In [ ]:
# default_exp preprocessing

# Preprocessing

> Tiling and such

In [ ]:
#hide
from nbdev.showdoc import *


In [ ]:
#export
from pathlib import Path
import os
import geopandas as gpd
import pandas as pd
from osgeo import gdal
from shapely.geometry import Point, Polygon, box
import numpy as np
from itertools import product
from typing import Tuple
from tqdm import tqdm
from fastcore.basics import *
from drone_detector.core import *
import json

## Tiler

In [ ]:
# export

class Tiler():
    "Similar functions than ´solaris.tile.raster_tile' but with suitable dependencies"
    def __init__(self, outpath=None, gridsize_x:int=640, gridsize_y:int=480, 
                 overlap:Tuple[int, int]=(100, 100)):
        store_attr()
        self.grid = None
        if outpath is not None and not os.path.exists(outpath):
            os.makedirs(outpath)
            os.makedirs(f'{outpath}/raster_tiles')
            os.makedirs(f'{outpath}/vector_tiles')
        
    def tile_raster(self, path_to_raster:str):
        self.grid = make_grid(str(path_to_raster), gridsize_x=self.gridsize_x,
                              gridsize_y=self.gridsize_y, overlap=self.overlap)
        raster = gdal.Open(path_to_raster)
        for row in tqdm(self.grid.itertuples()):
            tempraster = gdal.Translate(f'{self.outpath}/raster_tiles/{row.cell}.tif', raster,
                                        projWin=[row.geometry.bounds[0], row.geometry.bounds[3], 
                                                 row.geometry.bounds[2], row.geometry.bounds[1]]
                                       )
            tempraster = None
        return
    
    
    def tile_vector(self, path_to_vector:str):
        if self.grid is None:
            print('No raster grid specified, use Tiler.tile_raster to determine grid limits')
            return
        vector = gpd.read_file(path_to_vector)
        # God bless spatial index
        sindex = vector.sindex
        for row in tqdm(self.grid.itertuples()):
            possible_matches_index = list(sindex.intersection(row.geometry.bounds))
            tempvector = vector.iloc[possible_matches_index].copy()
            tempvector = gpd.clip(tempvector, row.geometry, keep_geom_type=True)
            # No annotations -> no shapefile
            if len(tempvector) == 0: continue
            
            # Filter too small geometries TODO
            tempvector.to_file(f'{self.outpath}/vector_tiles/{row.cell}.shp')
        return
    
def untile_vector(path_to_targets:str, outpath:str, non_max_suppression_thresh:float=0.0):
    "Create single shapefile from predictions"
    pred_files = [f for f in os.listdir(path_to_targets) if f.endswith('.shp')]
    gdf = None
    for p in tqdm(pred_files):
        temp_gdf = gpd.read_file(f'{path_to_targets}/{p}')
        if gdf is None: gdf = temp_gdf
        else: gdf = gdf.append(temp_gdf)
    print(f'{len(gdf)} polygons before non-max suppression')
    if non_max_suppression_thresh != 0:
        np_bounding_boxes = np.array([b.bounds for b in gdf.geometry])
        idxs = non_max_suppression_fast(np_bounding_boxes, overlap_thresh = non_max_suppression_thresh)
        gdf = gdf.iloc[idxs]
    print(f'{len(gdf)} polygons after non-max suppression')
    gdf.to_file(outpath)
    return

## COCOProcessor

In [ ]:
# export

class COCOProcessor():
    "Handles Transformations from shapefiles to COCO-format and backwards"
    
    def __init__(self, data_path:str, outpath:str, coco_info:dict, coco_licenses:list,
                 coco_categories:list):
        store_attr()
        self.raster_path = f'{self.data_path}/raster_tiles'
        self.vector_path = f'{self.data_path}/vector_tiles'
        
        self.coco_dict = {
            'info': coco_info,
            'licenses': coco_licenses,
            'images': [],
            'annotations': [],
            'categories': coco_categories,
            'segment_info': []
        }
        self.categories = {c['name']:c['id'] for c in self.coco_dict['categories']}
        
        
    def shp_to_coco(self, outfile:str='coco.json'):
        "Process shapefiles from self.vector_path to coco-format and save to self.outpath/outfile"
        vector_tiles = [f for f in os.listdir(self.vector_path) if f.endswith('.shp')]
        # If no annotations are in found in raster tile then there is no shapefile for that
        raster_tiles = [f'{fname[:-4]}.tif' for fname in vector_tiles]
        self.coco_dict['images'] = [{'file_name': raster_tiles[i],
                                     'id': i} for i in rangeof(raster_tiles)]
        ann_id = 0
        for i in tqdm(rangeof(raster_tiles)):
            gdf = gpd.read_file(f'{self.vector_path}/{vector_tiles[i]}')
            tfmd_gdf = gdf_to_px(gdf, f'{self.raster_path}/{raster_tiles[i]}')
            for row in tfmd_gdf.itertuples():
                category_id = self.categories[row.label]
                self.coco_dict['annotations'].append(_process_shp_to_coco(i, category_id, ann_id, row.geometry))
                ann_id += 1
        with open(f'{self.outpath}/{outfile}', 'w') as f: json.dump(self.coco_dict, f)

        return
    
    def coco_to_shp(self, coco_path:str=None, outdir:str='predicted_vectors'):
        "TODO handle multipolygons"
        if not os.path.exists(f'{self.outpath}/{outdir}'): os.makedirs(f'{self.outpath}/{outdir}')
        if coco_path is None: coco_path = f'{self.outpath}/coco.json'
        with open(coco_path) as f:
            coco_data = json.load(f)
        
        annotations = coco_data['annotations']
        images = coco_data['images']
        categories = coco_data['categories']
        for i in tqdm(images):
            anns_in_image = [a for a in annotations if a['image_id'] == i['id']]
            if len(anns_in_image) == 0: continue
            cats = []
            polys = []
            for a in anns_in_image:
                # Single polygon
                if len(a['segmentation']) == 1:
                    cats.append(a['category_id'])
                    xy_coords = [(a['segmentation'][0][i], a['segmentation'][0][i+1]) for i in range(0,len(a['segmentation'][0]),2)]
                    xy_coords.append(xy_coords[-1])
                    polys.append(Polygon(xy_coords))
                # Multipolygon TODO handle better
                else: 
                    for p in rangeof(a['segmentation']):
                        cats.append(a['category_id'])
                        xy_coords = [(a['segmentation'][p][i], a['segmentation'][p][i+1]) 
                                     for i in range(0,len(a['segmentation'][p]),2)]
                        xy_coords.append(xy_coords[-1])
                        polys.append(Polygon(xy_coords))
            gdf = gpd.GeoDataFrame({'label':cats, 'geometry':polys})
            tfmd_gdf = georegister_px_df(gdf, f'{self.raster_path}/{i["file_name"]}')
            tfmd_gdf.to_file(f'{self.outpath}/{outdir}/{i["file_name"][:-4]}.shp')
        return

def _process_shp_to_coco(image_id, category_id, ann_id, poly:Polygon):
    "TODO handle multipolygons"
    ann_dict = {
        'segmentation': [],
        'area': None, 
        'bbox': [],
        'category_id': category_id,
        'id' : ann_id,
        'image_id': image_id,
        'iscrowd': 0,
    }
    ann_dict['bbox'] = [(poly.bounds[0]), 
                        (poly.bounds[1]), 
                        (poly.bounds[2]-poly.bounds[0]), 
                        (poly.bounds[3]-poly.bounds[1])]
    ann_dict['area'] = poly.area
    if poly.type == 'Polygon':
        ann_dict['segmentation'] = [list(sum(poly.exterior.coords[:-1], ()))]
    elif poly.type == 'MultiPolygon':
        ann_dict['segmentation'] = [list(sum(p.exterior.coords[:-1], ())) for p in list(poly)]
    return ann_dict


## Icevision predictions to coco-json

In [ ]:
def process_preds_to_coco(samples:list, preds:list, categories:list=None) -> dict:
    coco_dict = {
            'info': None,
            'licenses': None,
            'images': [],
            'annotations': [],
            'categories': categories,
            'segment_info': []
        }    
    
    coco_dict['images'] =  [{'file_name': s['filepath'], 'id': s['imageid']} for s in samples]
    
    for i, (s, p) in enumerate(zip(samples, preds)):
        coco_dict['annotations'].append(_process_pred_to_coco(p, s['imageid'], i))
    return coco_dict

def _process_pred_to_coco(pred:dict, imageid:int, ann_id:int) -> dict:
    anns = []
    for i in rangeof(pred['masks']):
        ann_dict = {
            'segmentation': [],
            'area': None,
            'iscrowd': 0,
            'category_id': None,
            'id': ann_id,
            'image_id': imageid,
            'bbox': []
        }
        
        anns.append([ann_dict])
        
    return anns